## **Gerekli Kütüphanelerin Kurulumu ve Aktarılması**

* **Pandas**, **numpy** python'da veri işleme, veri okuma, sayısal işlemler ve matematiksel hesaplamalar için kullanılır.(Her projede olmazsa olmaz diyebileceğimiz kütüphanelerdir.)
* **Matplotlib**, **PIL** ve **Seaborn** kütüphanesi veri görselleştirme, grafiksel işlemlerde kullanılır.
* **Tensorflow**, modelimizi oluşturacağımız ve eğitim, test vb. bir sürü alanda kullanabileceğimiz geniş çalışma alanı sunan bir kütüphanedir.(Google tarafından geliştirilmiştir.)
* **sklearn**, bu projede etiketleri kategorileştirmek için ve veri setimizi ayırmak için kullandık.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import warnings
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout,BatchNormalization
import matplotlib.image as mpimg

Bazen matplotlib olsun bazı kısımlarda "warning" çıkıyor onu bu şekilde filtreleyerek göz önünde olmasını engelliyoruz.

In [ ]:
warnings.filterwarnings("ignore")

Veri setini çıkarma işlemi gerçekleştiriyoruz, label ve path olarak diziye atıyoruz ve DataFrame oluşturuyoruz.

In [ ]:
label= []
path= []
fish_dir= "/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset"
for dir_name,_,filenames in os.walk(fish_dir):
    for filename in filenames:
        if os.path.splitext(filename)[-1]== ".png":
            if dir_name.split()[-1] != "GT":
                label.append(os.path.split(dir_name)[-1])
                path.append(os.path.join(dir_name,filename))
                
data= pd.DataFrame(columns=["path","label"])
data["path"]= path
data["label"]= label

Oluşturduğumuz DataFrame hakkında bilgi alabilmek için "info()" kullandık. 

In [ ]:
data.info()

**"head()"** ile belirtilen sayıdaki veri ekrana yazdırılıyor.

In [ ]:
data.head(5)

## **Veri Seti Görselleştirme**

**1. unique_labels"** oluşturarak her bir sınıftan bir veri görüntüleme işlemi gerçekleştirdim.

In [ ]:
unique_labels = data.drop_duplicates(subset="label")

for i in range(len(unique_labels)):
    img_path = unique_labels["path"].iloc[i]
    label = unique_labels["label"].iloc[i]
    img = mpimg.imread(img_path)
    
    plt.figure()
    plt.imshow(img)
    plt.title(label)
    plt.axis("off")
    plt.show()

**2.** Label sayısını alarak kaç adet veri içerdiğini görselleştirdim

In [ ]:
label_counts= data["label"].value_counts()
colors = plt.cm.get_cmap('viridis', len(label_counts)) # her bir bar'ın farkli renkte olmasını istedim:)
plt.figure(figsize=(10,6))
label_counts.plot(kind='bar', color=[colors(i) for i in range(len(label_counts))])
plt.title("Number of Images per Label")
plt.xlabel("Labels")
plt.ylabel("Number of Images")

plt.xticks(rotation=45,ha="right") #Label'lar birbirini engellemesin diye 45 derece sağ döndürülecek.
plt.tight_layout() #Grafiklerin taşmasını engeller.
plt.show()

**One-Hot-Encoding** kullanarak etiketlerimizi kategorize ediyoruz

In [ ]:
encoder= OneHotEncoder(sparse=False)
labels_encoded= encoder.fit_transform(data["label"].values.reshape(-1,1))

In [ ]:
labels_encoded

Veri ayrıştırma ile veri setimizi eğitim için train-test olarak ayırıyoruz

In [ ]:
X_train,X_test,y_train,y_test= train_test_split(data["path"],labels_encoded,test_size=0.2,random_state=42)

## **Veri Ön İşleme(Image Preprocessing)**

In [1]:
img_size= (128,128)
def preprocessing_img(image_path):
    img=load_img(image_path,target_size=img_size)
    img_array= img_to_array(img)
    img_array= img_array / 255.0 #normalization işlemi
    return img_array
    
#X_train_processed= np.array([preprocessing_img(img) for img in X_train])
#X_test_processed= np.array([preprocessing_img(img) for img in X_test]#) ## hata aldı cpu yetersiz kaldığı için bende thread yöntemi ile çözüm buldum.

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def preprocess_images_multithread(img_paths):
    with ThreadPoolExecutor() as executor:
        processed_images = list(executor.map(preprocessing_img, img_paths))
    return np.array(processed_images)

X_train_processed = preprocess_images_multithread(X_train)
X_test_processed = preprocess_images_multithread(X_test)

## **Model Oluşturma**

Modeli oluştururken pek çok farklı kombinasyon denedim ve araştırdım. Sürem kısıtlı olduğu için, en iyi sonuç aldığım modeli aşağıda görebilirsiniz. İlk başta her katmana Dropout eklemeyi düşündüm, ancak bu yöntem iyi sonuçlar vermedi. En iyi sonucu, yalnızca tek bir Dropout katmanı ekleyerek elde ettim. Aktivasyon fonksiyonu olarak “relu” kullandım; bu fonksiyon, negatif değerleri bile 0-1 aralığına getirerek etkili sonuçlar sağlıyor. Çıkış katmanına, etiket sayısı kadar nöron ekledim ve aktivasyon olarak “softmax” kullandım.

In [ ]:
model = Sequential()

# Giriş katmanı (Flatten) - Görüntüleri tek bir vektör haline getirme
model.add(Flatten(input_shape=(128, 128, 3)))

model.add(Dense(512, activation='relu'))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))  
model.add(Dense(128, activation='relu'))

model.add(Dense(128, activation='relu'))

model.add(Dense(labels_encoded.shape[1], activation='softmax'))


In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

* optimizer=Adam(learning_rate=1e-4): Modelin ağırlıklarını güncellemek için Adam optimizasyon algoritmasını kullanıldı. Öğrenme oranı **1e-4 (0.0001)** olarak ayarlanmış, bu nispeten düşük bir öğrenme oranıdır, modelin daha yavaş ve hassas öğrenmesini sağlar.

* loss='categorical_crossentropy': Kategorik sınıflandırma problemlerinde kullanılan bir kayıp fonksiyonudur. Eğer model çoklu sınıflandırma yapıyorsa (örneğin, birden fazla etiket veya kategori varsa), bu fonksiyon doğru bir seçimdir.

* metrics=['accuracy']: Modelin performansını değerlendirirken doğruluk metriği kullandım. Bu, eğitim ve test aşamalarında modelin doğruluğunu gözlemlemek için kullanılır.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

Early Stopping aşırı öğrenmeyi(overfitting) engellemek adına burada tanımladık 3 adımdada bozulma olursa early stopping yapıyor.

In [ ]:
history = model.fit(
    X_train_processed,
    y_train,
    epochs=30,          
    validation_data=(X_test_processed, y_test),
    callbacks=[early_stopping]
    
)

Modelin eğitimini başlatıyoruz ve bir değişkene kaydediyoruz ki sonrasında analizler gerçekleştirebilelim.

In [ ]:
# Test seti üzerinde performans değerlendirme
test_loss, test_acc = model.evaluate(X_test_processed, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_acc}')

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Modelin tahminleri
y_pred = model.predict(X_test_processed)
y_pred_classes = np.argmax(y_pred, axis=1)  # Tahmin edilen sınıfları al
y_true_classes = np.argmax(y_test, axis=1)  # Gerçek sınıflar

# Confusion matrix hesaplama
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)

# Confusion Matrix'i görselleştirme
plt.figure(figsize=(10,7))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=encoder.categories_[0], yticklabels=encoder.categories_[0])
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title("Confusion Matrix")
plt.show()



Modelmizi kullanarak X_text olarka ayırdığımız veriler üzerinde tahmin gerçekleştiriyoruz ve Confusion Matrix ile görselleştirme yapıyoruz.

In [ ]:

# Classification Report - precision, recall, f1-score
print("Classification Report:")
print(classification_report(y_true_classes, y_pred_classes))


In [ ]:
# F1 score
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
print(f'F1 Score: {f1}')

In [ ]:
# Yanlış tahmin edilen indeksleri bulma
incorrect_indices = np.where(y_pred_classes != y_true_classes)[0]

# Yanlış tahmin edilen ilk birkaç örneği görselleştirme
num_examples = 5  # Kaç tane yanlış tahmin edilen görüntü sayısı
plt.figure(figsize=(15, 10))
for i, idx in enumerate(incorrect_indices[:num_examples]):
    plt.subplot(1, num_examples, i + 1)
    plt.imshow(X_test_processed[idx].reshape(128, 128, 3))  # Görüntülerin boyutunu ayarlayın
    plt.title(f"True: {encoder.categories_[0][y_true_classes[idx]]}\nPred: {encoder.categories_[0][y_pred_classes[idx]]}")
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Doğru tahmin edilen indeksleri bulma
correct_indices = np.where(y_pred_classes == y_true_classes)[0]

# Doğru tahmin edilen ilk birkaç örneği görselleştirme
plt.figure(figsize=(15, 10))
for i, idx in enumerate(correct_indices[:num_examples]):
    plt.subplot(1, num_examples, i + 1)
    plt.imshow(X_test_processed[idx].reshape(128, 128, 3))  # Görüntülerin boyutunu ayarlayın
    plt.title(f"True: {encoder.categories_[0][y_true_classes[idx]]}\nPred: {encoder.categories_[0][y_pred_classes[idx]]}")
    plt.axis('off')

plt.tight_layout()
plt.show()